# Multiagent systems - TSP Problem

In [1]:
import tsplib95
from random import randrange, random, shuffle
from bisect import bisect
import pandas as pd
import matplotlib.pyplot as plt
# import pixiedust
import numpy as np

In [2]:
#!pip install ipython-autotime
%load_ext autotime

## Load TSP Problem

In [3]:
# Data Source : http://www.math.uwaterloo.ca/tsp/world/countries.html
#TSP_FILE = 'uy734.tsp' # 🇺🇾 Uruguay
#TSP_FILE = 'wi29.tsp' # 🇪🇭 Western Sahara
TSP_FILE = 'ca4663.tsp' # 🇨🇦 Canada

time: 467 µs


In [4]:
# LOAD TSP FILE
PROBLEM = tsplib95.load(TSP_FILE)
print(type(PROBLEM))

<class 'tsplib95.models.StandardProblem'>
time: 43.9 ms


In [5]:
PROBLEM.type

'TSP'

time: 4.12 ms


In [6]:
N = len(list(PROBLEM.get_nodes())) # N is total cities
print("%s Cities"%N)

4663 Cities
time: 1.75 ms


In [7]:
# Example city coordinates
PROBLEM.node_coords[3]

[41983.3333, 82933.3333]

time: 1.97 ms


In [8]:
# Distance between first and last cities
edge = {'start':1,'end':N}
PROBLEM.get_weight(**edge)

45511

time: 2.02 ms


## Basic Ant

In [9]:
class Ant:
    tour = None
    tsp = PROBLEM
    
    def __init__(self, city_i=0):
        self.tour = []
        if city_i>0:
            self.visit(city_i)
    
    @property
    def current_city(self):
        return self.tour[-1]

    @property
    def tour_weight(self):
        return self.tsp.trace_tours([self.tour])[0]
    
    def visit(self, i:int):
        if i in self.tour and i != self.tour[0]:
            raise Exception("The city i: %s is already visited. Imposible to visit again"%i)
        if i < 1 or i > N:
            raise Exception("The city i (%s) is out of range: -> [1, %s]"%(i, N))
        self.tour.append(i)
    
    def distance_to(self, city_j:int):
        return self.tsp.get_weight(self.current_city, city_j)
    
    def _not_visited_cities(self):
        return [i for i in range(1,N+1) if i not in self.tour]
    
    def _raw_probability(self, city_j:int, pheromones_matrix):
        ## ASSUMPTION: We consider the edge has two ways. Phromones to go and to go back. In other words. I->J != J->I
        # careful, we must substract one from the cities index
        return (pheromones_matrix[self.current_city-1][city_j-1]**ALPHA) * ((1/self.distance_to(city_j))**BETA)
    
    def normalized_probabilities(self, pheromones_matrix):
        """ Returns a tuple
            First element: List of neighbors, cities not visited
            Second element: List of probabilities calculated with the formular of tau_ij^A* h_ij^B
        """
        neighbors = self._not_visited_cities()
        neighbors_pheromone_list = [self._raw_probability(neighbor_j, pheromones_matrix) for neighbor_j in neighbors]
        total = sum(neighbors_pheromone_list)
        normalized_probabilities = [pheromone_ij/total for pheromone_ij in neighbors_pheromone_list]
        #print(normalized_probabilities)
        return neighbors, normalized_probabilities
        
    def pick_next_city(self, cities, probabilities):
        roulette_x = random()
        idx = 0
        roulette_sum = 0
        for p in probabilities:
            roulette_sum += p
            if roulette_sum >= roulette_x  :
                return cities[idx]
            idx += 1
    
    def finished_tour(self):
        return len(self.tour) == N

    def plot_hot_map(self):
        df = pd.DataFrame([[0 for j in N] for i in N])
        for posi_i in range(1, len(ant.tour)):
            i = ant.tour[posi_i-1]-1
            j = ant.tour[posi_i]-1
            pheromones_to_add[i][j] += tau_delta
            
        plt.imshow(df, cmap='hot', interpolation='nearest')
        plt.show()

time: 6.73 ms


In [10]:
a = Ant(1)
print(a.tour)
a.visit(29)
print(a.tour)
print("Total weight of this ant tour is: %s"%a.tour_weight)

[1]
[1, 29]
Total weight of this ant tour is: 1840
time: 1.05 ms


In [11]:
def plot_pheromones(df, step, show=True, title=''):
    print(title)
    plt.imshow(df, cmap='hot', interpolation='nearest')
    plt.savefig("pheromones-%03d.png"%step)
    plt.show()
    #plt.imsave("pheromones-%03d.png"%step, df, cmap='hot')

time: 929 µs


## BASE LINE

In [12]:
# Solution joining all the cities in sequence
ant = Ant(1)
for i in range(2,N+1):
    ant.visit(i)
print(ant.tour)
print(ant.tour_weight)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

47892988
time: 140 ms


In [13]:
# Random Solution
ant = Ant(1)
random_cities = list(range(2,N+1))
shuffle(random_cities)
for i in random_cities:
    ant.visit(i)
print(ant.tour)
print(ant.tour_weight)

[1, 63, 127, 4560, 2583, 1876, 4412, 3109, 3290, 215, 3225, 2946, 2372, 753, 208, 2031, 3780, 1550, 2634, 823, 1743, 4567, 1706, 2000, 1032, 3898, 501, 2179, 3993, 3400, 4508, 1799, 8, 200, 4303, 4570, 3193, 3008, 1052, 1849, 905, 3717, 2925, 960, 1785, 3442, 4596, 1116, 3879, 4455, 2484, 904, 620, 3454, 2263, 3453, 3414, 703, 3276, 2920, 37, 1692, 1711, 747, 1663, 2422, 1752, 1576, 1914, 545, 4288, 1999, 154, 1852, 2050, 3915, 1100, 3435, 3809, 3738, 4341, 2690, 2896, 4148, 819, 824, 2319, 1282, 2756, 122, 3689, 3805, 3496, 1349, 1630, 4420, 1989, 1929, 3481, 1598, 3924, 51, 2164, 148, 1722, 1965, 4315, 2686, 79, 2498, 1540, 4597, 4660, 514, 1354, 2193, 4331, 2499, 1365, 2312, 1700, 2852, 889, 236, 429, 711, 1087, 151, 552, 4299, 3085, 724, 4587, 1094, 1414, 4374, 3570, 4532, 2595, 12, 816, 1753, 1121, 2989, 4631, 3344, 2585, 3788, 1831, 2090, 1529, 2612, 3843, 3643, 883, 3560, 2351, 1022, 4052, 4380, 626, 1596, 1826, 231, 1244, 3289, 2533, 4308, 1400, 1874, 563, 1787, 341, 2998, 2172

In [14]:
# Solution using the heuristic
ant = Ant(1)
while not ant.finished_tour():
    neighbors = ant._not_visited_cities()
    distances = []
    for city_j in range(1, len(neighbors)+1):
        distances.append(ant.distance_to(city_j))
    pos_min_distance = distances.index(min(distances))
    next_closest_city = neighbors[pos_min_distance]
    ant.visit(next_closest_city)
print(ant.tour)
print(ant.tour_weight)

[1, 2, 4, 7, 11, 16, 22, 29, 37, 46, 56, 67, 79, 92, 106, 121, 137, 154, 172, 191, 211, 232, 254, 277, 301, 326, 352, 379, 407, 436, 466, 497, 529, 562, 596, 631, 667, 704, 742, 781, 821, 862, 904, 947, 991, 1036, 1082, 1129, 1177, 1226, 1276, 1327, 1379, 1432, 1486, 1541, 1597, 1654, 1712, 1771, 1831, 1892, 1954, 2017, 2081, 2146, 2212, 2279, 2347, 2416, 2486, 2557, 2629, 2702, 2776, 2851, 2927, 3004, 3082, 3161, 3241, 3322, 3404, 3487, 3571, 3656, 3742, 3829, 3917, 4006, 4096, 4187, 4279, 4372, 4466, 4561, 4657, 4654, 4661, 4662, 4651, 4660, 4643, 4628, 4627, 4655, 4639, 4631, 4656, 4653, 4644, 4625, 4633, 4658, 4663, 4573, 4649, 4646, 4600, 4645, 4630, 4652, 4641, 4650, 4616, 4659, 4648, 4647, 4617, 4574, 4602, 4620, 4608, 4607, 4636, 4637, 4604, 4575, 4624, 4640, 4611, 4576, 4615, 4632, 4634, 4614, 4642, 4629, 4622, 4577, 4638, 4588, 4589, 4635, 4619, 4610, 4613, 4621, 4603, 4606, 4626, 4549, 4598, 4564, 4597, 4569, 4601, 4550, 4609, 4594, 4618, 4582, 4599, 4566, 4612, 4535, 4605, 

## ANT SYSTEM

In [15]:
def ant_system():
    # INIT MATRIX for each CITY IJ with TAU INITIAL (t_0)
    _pheromones_row = [TAU_INITIAL for i in range(N)]
    pheromones_matrix = [_pheromones_row for j in range(N)]

    history_list = []

    for step in range(STEPS):
        ants_list = []
        for ant_i in range(M_ANTS):
            # pick a starting point
            first_random_city = randrange(N)+1
            ant = Ant(first_random_city)
            ants_list.append(ant)
            while not ant.finished_tour():
                # calculate probability P_j for all unvisited neightbors J
                    # ANT SYSTEM (AS): Probability of each edge in the neighborhood
                    # p_ij_k = (t_ij^a * (1/d_ij)^b ) / SUM(all feasible g edges) # It's like edge normalized
                neighbors, probabilities = ant.normalized_probabilities(pheromones_matrix) # sum(probabilities) == 1
                # pick the next node using the probabilities
                next_city = ant.pick_next_city(neighbors, probabilities)
                ant.visit(next_city)
            ant.visit(first_random_city) # Close cycle??
        history_list.append(ants_list.copy()) # save results
        # update pheromone values based upon the quality of each solution
            # ANT SYSTEM (AS): All ants contribute updating the pheromone as follows
            # TAU_I_J = (1-RO)*TAU_I_J + SUM(Q/(Lk or 0)) # Attention! In TSP Lk will be always the same == N Total cities
                                               # Probably in TSP the length means the distance
        pheromones_to_add = [[0 for i in range(N)] for j in range(N)]
        for ant in ants_list:
            tau_delta = Q/ant.tour_weight
            for tour_i in range(1, len(ant.tour)):
                i = ant.tour[tour_i-1]-1 # city
                j = ant.tour[tour_i]-1 # next city
                pheromones_to_add[i][j] += tau_delta
        # update fermonones
        df = pd.DataFrame(pheromones_matrix)*(1-RO)+pd.DataFrame(pheromones_to_add)
        pheromones_matrix = df.values
        # PLOT every 10th of steps
        if step % int(STEPS/100) == 0:
            plot_pheromones(df,step=step+1, title="Step %s from %s."%(step+1,STEPS))
    # Plot last result
    plot_pheromones(df,step=step+1, title="Step %s from %s."%(step+1,STEPS))
    return history_list, ants_list

time: 3.31 ms


In [ ]:
M_ANTS = int(N) # Number of ants ~ to number of nodes (N)
ALPHA = 1 # History coefficietn ~ 1
BETA = 3 # 0,1,2,3,4,5,6 # Heuristic Coefficient [2,5]
RO = 0.2# Evaporation rate # It's like cooling. A high value is similar to very decrease the temparature drastically and get stucked in a local optimum
Q = 1*30000 # Pheromone change factor
TAU_INITIAL = 1/70000 # Initial pheromone ~ 1/RO*C^nn ; C^nn is the length of the tour generated by the nearest neighbor heuristic
STEPS = 50

history_list, ants_list = ant_system()

In [ ]:
tours_weight_list = [a.tour_weight for a in ants_list]
print(tours_weight_list)

In [ ]:
#pos_min = tours_weight_list.index(min(tours_weight_list))
all_ants_list = [ant for ants_step_list in history_list for ant in ants_step_list]
all_tours_weight_list = [a.tour_weight for a in all_ants_list]
pos_min = all_tours_weight_list.index(min(all_tours_weight_list))
print("Min weigth: %s"%all_tours_weight_list[pos_min])
best_ant = all_ants_list[pos_min]
best_tour = best_ant.tour
print("Best Tour: %s"%best_tour)

In [ ]:
pd.DataFrame(all_tours_weight_list).plot(figsize=(15,10))

In [ ]:
pd.DataFrame([min([a.tour_weight for a in ants_step_list]) for ants_step_list in history_list]).plot()

In [ ]:
pd.DataFrame([max([a.tour_weight for a in ants_step_list]) for ants_step_list in history_list]).plot()

In [ ]:
pd.DataFrame([np.mean([a.tour_weight for a in ants_step_list]) for ants_step_list in history_list]).plot()

In [ ]:
map_df_lat = pd.DataFrame([PROBLEM.node_coords[i][0] for i in best_tour], columns=['lat'])
map_df_long = pd.DataFrame([PROBLEM.node_coords[i][1] for i in best_tour], columns=['long'])*-1
print(len(map_df_lat))
plt.figure(figsize=(15,10))
plt.plot(map_df_long,
              map_df_lat,
              c='DarkBlue',
              #style=['o', 'rx'],
              #s=2,
              #figsize=(15,8),
              marker="o",
              markerfacecolor="r")

In [ ]:
best_ant.tour_weight